In [ ]:
import json
import os
import pandas
import pyarrow
import sys

from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
import create_queries
import prepare_data
%load_ext autoreload
%autoreload 2

In [ ]:
pandas.set_option('display.max_rows', 50)

In [ ]:
COMPANY_IDENTIFIER = 'CSC'
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'

In [ ]:
company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
    product_name='Camino Gummies Midnight Blueberry',
)
company_outgoing_transfer_packages_query = create_queries.create_company_outgoing_transfer_packages_query(
    COMPANY_IDENTIFIER,
    TRANSFER_PACKAGES_START_DATE,
    product_name='Camino Gummies Midnight Blueberry',
)
company_inventory_packages_query = create_queries.create_company_inventory_packages_query(
    COMPANY_IDENTIFIER,
    include_quantity_zero=True,
    product_name='Camino Gummies Midnight Blueberry',
)
company_sales_transactions_query = create_queries.create_company_sales_transactions_by_product_name(
    COMPANY_IDENTIFIER,
    product_name='Camino Gummies Midnight Blueberry',
)
company_incoming_transfer_packages_dataframe = pandas.read_sql_query(company_incoming_transfer_packages_query, engine)
company_outgoing_transfer_packages_dataframe = pandas.read_sql_query(company_outgoing_transfer_packages_query, engine)
company_inventory_packages_dataframe = pandas.read_sql_query(company_inventory_packages_query, engine)
company_sales_transactions_dataframe = pandas.read_sql_query(company_sales_transactions_query, engine)

In [ ]:
company_incoming_transfer_packages_dataframe = company_incoming_transfer_packages_dataframe[[
    'delivery_type',
    'created_date',
    'received_datetime',
#     'shipment_transaction_type',
#     'shipper_facility_license_number', 'shipper_facility_name',
#     'recipient_facility_license_number', 'recipient_facility_name',
#     'shipment_type_name', 'shipment_transaction_type_1',
    'package_id',
    'package_label',
#     'type', 'source_package_labels',
#     'source_harvest_names', 'shipment_package_state', 'is_testing_sample',
#     'is_trade_sample',
    'product_category_name',
    'product_name',
#     'package_lab_results_status', 'shipper_wholesale_price',
#     'shipped_quantity', 'shipped_unit_of_measure',
    'receiver_wholesale_price',
    'received_quantity',
    'received_unit_of_measure',
#     'receiver_wholesale_price_1',
#     'item_unit_weight', 'item_unit_weight_unit_of_measure_name'
]]

In [ ]:
company_incoming_transfer_packages_dataframe

In [ ]:
company_outgoing_transfer_packages_dataframe.columns

In [ ]:
company_outgoing_transfer_packages_dataframe = company_outgoing_transfer_packages_dataframe[[
    'delivery_type',
    'license_number',
#     'manifest_number',
    'created_date',
    'received_datetime',
#     'shipper_facility_license_number',
#     'shipper_facility_name', 'recipient_facility_license_number',
#     'recipient_facility_name', 'shipment_type_name',
#     'shipment_transaction_type',
    'package_id',
    'package_label',
#     'type',
#     'source_package_labels', 'source_harvest_names',
#     'shipment_package_state', 'is_testing_sample', 'is_trade_sample',
#     'product_category_name', 'product_name', 'package_lab_results_status',
#     'shipper_wholesale_price', 'shipped_quantity',
#     'shipped_unit_of_measure',
    'received_quantity',
    'received_unit_of_measure',
    'receiver_wholesale_price',
#     'item_unit_weight', 'item_unit_weight_unit_of_measure_name'
]]

In [ ]:
company_outgoing_transfer_packages_dataframe

In [ ]:
incoming_package_ids_set = set(list(company_incoming_transfer_packages_dataframe['package_id']))
inventory_package_ids_set = set(list(company_inventory_packages_dataframe['package_id']))
sales_transaction_package_ids_set = set(list(company_sales_transactions_dataframe['package_id']))
len(incoming_package_ids_set), len(inventory_package_ids_set), len(sales_transaction_package_ids_set)

In [ ]:
company_inventory_packages_dataframe

In [ ]:
# company_inventory_packages_dataframe = company_inventory_packages_dataframe.merge(
#     company_incoming_transfer_packages_dataframe,
#     on='package_id',
#     how='inner',
#     suffixes=('_l', '_r'),
# )
company_inventory_packages_dataframe = company_inventory_packages_dataframe.sort_values('received_datetime', ascending=False)
company_inventory_packages_dataframe.columns

In [ ]:
company_inventory_packages_dataframe = company_inventory_packages_dataframe[[
    'package_id',
    'packaged_date',
    'received_datetime',
    'quantity',
    'received_quantity',
    'receiver_wholesale_price',
    'received_unit_of_measure',
]]
company_inventory_packages_dataframe.head(50)

In [ ]:
company_inventory_packages_dataframe['received_quantity'].sum() - company_inventory_packages_dataframe['quantity'].sum()

In [ ]:
company_sales_transactions_dataframe

In [ ]:
company_sales_transactions_dataframe['quantity_sold'].sum()

In [ ]:
for sales_transaction_package_id in list(sales_transaction_package_ids_set):
    if sales_transaction_package_id not in inventory_package_ids_set:
        print(sales_transaction_package_id)

In [ ]:
company_inventory_package_records = company_inventory_packages_dataframe.to_dict('records')
company_sales_transaction_records = company_sales_transactions_dataframe.to_dict('records')

In [ ]:
company_inventory_package_records[:1], company_sales_transaction_records[:1]

In [ ]:
company_inventory_package_tuples = [('incoming', r['received_datetime'], r['package_id'], r['received_quantity']) for r in company_inventory_package_records]
company_inventory_package_tuples[:2]

In [ ]:
company_sales_transaction_tuples = [('sale', r['sales_datetime'], r['package_id'], r['quantity_sold']) for r in company_sales_transaction_records]
company_sales_transaction_tuples[:2]

In [ ]:
mixed_tuples = company_inventory_package_tuples + company_sales_transaction_tuples
mixed_tuples = sorted(mixed_tuples, key=lambda t: t[1])

In [ ]:
for mixed_tuple in mixed_tuples:
    print(mixed_tuple)